In [ ]:
!curl -sSL "https://julialang-s3.julialang.org/bin/linux/x64/1.5/julia-1.5.1-linux-x86_64.tar.gz" -o julia.tar.gz
!tar -xzf julia.tar.gz -C /usr --strip-components 1
!rm -rf julia.tar.gz*
!julia -e 'using Pkg; pkg"add IJulia"'

 Installing known registries into `~/.julia`
######################################################################## 100.0%
      Added registry `General` to `~/.julia/registries/General`
  Resolving package versions...
  Installed Conda ─────────── v1.5.2
  Installed libsodium_jll ─── v1.0.19+0
  Installed MbedTLS_jll ───── v2.16.8+1
  Installed VersionParsing ── v1.2.0
  Installed Preferences ───── v1.2.2
  Installed Parsers ───────── v1.1.0
  Installed IJulia ────────── v1.23.2
  Installed JLLWrappers ───── v1.3.0
  Installed ZMQ ───────────── v1.2.1
  Installed SoftGlobalScope ─ v1.1.0
  Installed MbedTLS ───────── v1.0.3
  Installed JSON ──────────── v0.21.1
  Installed Artifacts ─────── v1.3.0
  Installed ZeroMQ_jll ────── v4.3.2+6
  Installed TOML ──────────── v1.0.3
######################################################################## 100.0%
######################################################################## 100.0%
######################################################

上記をshift+enterで実行後、「ランタイム」->「ランタイムのタイプを変更」->「保存」を押してください。

In [ ]:
versioninfo()

Julia Version 1.5.1
Commit 697e782ab8 (2020-08-25 20:08 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, broadwell)


# GPUの確認

In [ ]:
import Pkg; Pkg.add("CUDAdrv")

   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
  Installed CUDAapi ─ v4.0.0
  Installed CEnum ─── v0.3.0
  Installed CUDAdrv ─ v6.3.0
Updating `~/.julia/environments/v1.5/Project.toml`
  [c5f51814] + CUDAdrv v6.3.0
Updating `~/.julia/environments/v1.5/Manifest.toml`
  [fa961155] + CEnum v0.3.0
  [3895d2a7] + CUDAapi v4.0.0
  [c5f51814] + CUDAdrv v6.3.0


In [ ]:
# Google Colab上で使用しているGPUマシン名を取得
using CUDAdrv; CUDAdrv.name(CuDevice(0))

┌ Info: Precompiling CUDAdrv [c5f51814-7f29-56b8-a69c-e4d8f6be1fde]
└ @ Base loading.jl:1278


"Tesla T4"

# パッケージを追加

In [ ]:
import Pkg; Pkg.add("Flux")
import Pkg; Pkg.add("MLDatasets")
import Pkg; Pkg.add("CuArrays")
import Pkg; Pkg.add("ImageCore")

   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
  Installed ZygoteRules ────────────────── v0.2.1
  Installed CodecZlib ──────────────────── v0.7.0
  Installed CompilerSupportLibraries_jll ─ v0.3.4+0
  Installed StatsAPI ───────────────────── v1.0.0
  Installed TimerOutputs ───────────────── v0.5.9
  Installed NaNMath ────────────────────── v0.3.5
  Installed ExprTools ──────────────────── v0.1.3
  Installed Flux ───────────────────────── v0.12.1
  Installed AbstractTrees ──────────────── v0.3.4
  Installed Zlib_jll ───────────────────── v1.2.11+18
  Installed DiffResults ────────────────── v1.0.3
  Installed DataAPI ────────────────────── v1.6.0
  Installed Colors ─────────────────────── v0.12.8
  Installed Media ──────────────────────── v0.5.0
  Installed CommonSubexpressions ───────── v0.3.0
  Installed IRTools ────────────────────── v0.4.2
  Installed Juno ───────────────────────── v0.8.4
  Installed DocStringExtensions ────────── v0.8.4
  Inst

# パッケージを呼び出し

In [ ]:
using Flux
using Flux.Data: DataLoader
using Flux: @epochs, onehotbatch, onecold, logitcrossentropy, train!, throttle, flatten
using MLDatasets
using CuArrays
using ImageCore

┌ Info: Precompiling Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1278


######################################################################### 100.0%


######################################################################### 100.0%


######################################################################### 100.0%


┌ Info: Precompiling MLDatasets [eb30cadb-4394-5ae3-aed4-317e484a6458]
└ @ Base loading.jl:1278


# データ読み込み

In [ ]:
# 学習データ
train_x, train_y = MLDatasets.MNIST.traindata(Float32)
# テストデータ
test_x, test_y = MLDatasets.MNIST.testdata(Float32)

# ワンホットベクトルのバッチを作成
train_y, test_y = onehotbatch(train_y, 0:9), onehotbatch(test_y, 0:9)

batch_size, shuffle = 128, true

# ミニバッチを作成
train_data = DataLoader(train_x, train_y, batchsize=batch_size, shuffle=shuffle)
test_data = DataLoader(test_x, test_y, batchsize=batch_size, shuffle=shuffle)

This program has requested access to the data dependency MNIST.
which is not currently installed. It can be installed automatically, and you will not see this message again.

Dataset: THE MNIST DATABASE of handwritten digits
Authors: Yann LeCun, Corinna Cortes, Christopher J.C. Burges
Website: http://yann.lecun.com/exdb/mnist/

[LeCun et al., 1998a]
    Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner.
    "Gradient-based learning applied to document recognition."
    Proceedings of the IEEE, 86(11):2278-2324, November 1998

The files are available for download at the offical
website linked above. Note that using the data
responsibly and respecting copyright remains your
responsibility. The authors of MNIST aren't really
explicit about any terms of use, so please read the
website to make sure you want to download the
dataset.



Do you want to download the dataset from ["https://ossci-datasets.s3.amazonaws.com/mnist/train-images-idx3-ubyte.gz", "https://ossci-datasets.s3.amazonaws.com/mn

DataLoader((Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]), 128, 10000, true, 10000, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  9991, 9992, 9993, 9994, 9995, 9996, 9997, 9998, 9999, 10000], true)

In [ ]:
# 手書き数字は28×28サイズの画像データとなっている
MLDatasets.MNIST.convert2image(MLDatasets.MNIST.traintensor(1))

In [ ]:
# 上記の画像を数値で表したもの、正規化されており、0～1の間で色の濃淡を表している
MLDatasets.MNIST.traintensor(1)

28×28 reinterpret(N0f8, ::Array{UInt8,2}):
 0.0  0.0  0.0  0.0  0.0  0.0    0.0    …  0.0    0.0    0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0    0.0       0.0    0.0    0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0    0.0       0.0    0.0    0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0    0.0       0.0    0.0    0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0    0.0       0.0    0.216  0.533  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0    0.0    …  0.0    0.675  0.992  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0    0.0       0.071  0.886  0.992  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0    0.0       0.671  0.992  0.992  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0    0.118     0.859  0.992  0.831  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0    0.141     0.992  0.992  0.529  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0    0.369  …  0.992  0.992  0.518  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0    0.604     0.992  0.957  0.063  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  

# モデル定義

In [ ]:
model = Chain(
        flatten,
        Dense(28^2, 32, relu),
        Dropout(0.2),
        Dense(32, 10)
)

Chain(flatten, Dense(784, 32, relu), Dropout(0.2), Dense(32, 10))

In [ ]:
# パラメータ
params = Flux.params(model)

In [ ]:
# 最適化関数
optimiser = ADAM()

In [ ]:
# 損失関数
loss(x,y) = logitcrossentropy(model(x), y) 

In [ ]:
# コールバック関数
evalcb = () -> @show(loss(train_x, train_y))

In [ ]:
params = Flux.params(model) # パラメータ

optimiser = ADAM() # 最適化関数
loss(x,y) = logitcrossentropy(model(x), y) # 損失関数

evalcb = () -> @show(loss(train_x, train_y)) # コールバック関数

#1 (generic function with 1 method)

# 学習

In [ ]:
num_epochs = 10
@epochs num_epochs train!(loss, params, train_data, optimiser, cb = throttle(evalcb, 5))

┌ Info: Epoch 1
└ @ Main /root/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


loss(train_x, train_y) = 2.336897f0


┌ Info: Epoch 2
└ @ Main /root/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


loss(train_x, train_y) = 0.30005422f0


┌ Info: Epoch 3
└ @ Main /root/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


...

loss(train_x, train_y) = 0.13444702f0


┌ Info: Epoch 9
└ @ Main /root/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


loss(train_x, train_y) = 0.1262761f0


┌ Info: Epoch 10
└ @ Main /root/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


loss(train_x, train_y) = 0.11723247f0


# テスト

In [ ]:
# 正解率の算出
function accuracy(data_loader, model)
    acc_correct = 0
    for (x, y) in data_loader
        batch_size = size(x)[end]
        acc_correct += sum(onecold(model(x)) .== onecold(y)) / batch_size
    end
    return acc_correct / length(data_loader)
end

accuracy (generic function with 1 method)

In [ ]:
# モデルをテストモードの変更
testmode!(model)
accuracy(test_data, model)

0.9581685126582279

# 速度計測

In [ ]:
import Pkg; Pkg.add("BenchmarkTools")

  Resolving package versions...
  Installed BenchmarkTools ─ v1.0.0
Updating `~/.julia/environments/v1.5/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.0.0
Updating `~/.julia/environments/v1.5/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.0.0


In [ ]:
using BenchmarkTools

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1278


In [ ]:
# @benchmarkを先頭に付与することで、自動的にコードの実行時間を計算してくれる
num_epochs = 10
@benchmark @epochs num_epochs train!(loss, params, train_data, optimiser, cb = throttle(evalcb, 5))

┌ Info: Epoch 1
└ @ Main /root/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


loss(train_x, train_y) = 2.263474f0


┌ Info: Epoch 2
└ @ Main /root/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


loss(train_x, train_y) = 0.29224166f0


┌ Info: Epoch 3
└ @ Main /root/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


...

┌ Info: Epoch 9
└ @ Main /root/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


loss(train_x, train_y) = 0.051090453f0


┌ Info: Epoch 10
└ @ Main /root/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


loss(train_x, train_y) = 0.053101595f0


BenchmarkTools.Trial: 
  memory estimate:  6.07 GiB
  allocs estimate:  1800267
  --------------
  minimum time:     9.431 s (4.99% GC)
  median time:      9.431 s (4.99% GC)
  mean time:        9.431 s (4.99% GC)
  maximum time:     9.431 s (4.99% GC)
  --------------
  samples:          1
  evals/sample:     1